# 一、用Pandas读入数据

In [ ]:
import pandas as pd

In [ ]:
# read CSV file directly from a URL and save the results
data = pd.read_csv('Advertising.csv', index_col=0)

In [ ]:
# display the first 5 rows
data.head()

In [ ]:
# display the last 5 rows
data.tail()

In [ ]:
# check the shape of the DataFrame (rows, columns)
data.shape

共有200个观测到的样例（每行一个样例），每个样例是一个市场。

每个市场由三个特征描述：

TV：花在电视广告上的费用

radio：花在广播广告上的费用

newspaper：花在报纸广告上的费用

每个市场待预测的值是：

sales：商品的销售量

因为销售量是一个连续值，所以此预测问题是一个回归问题。

# 二、用seaborn可视化数据

In [ ]:
# conventional way to import seaborn
import seaborn as sns

# allow plots to appear within the notebook
%matplotlib inline

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.pairplot(data, x_vars=['TV','radio','newspaper'], y_vars='sales', height=7, aspect=0.7)

# 三、线性回归

$y = \beta_0 + \beta_1 \times TV + \beta_2 \times Radio + \beta_3 \times Newspaper$

## 用Pandas准备X和y

In [ ]:
# create a Python list of feature names
feature_cols = ['TV', 'radio', 'newspaper']

# use the list to select a subset of the original DataFrame
X = data[feature_cols]

# equivalent command to do this in one line
X = data[['TV', 'radio', 'newspaper']]

# print the first 5 rows
X.head()

In [ ]:
# check the type and shape of X
print(type(X))
print(X.shape)

In [ ]:
# select a Series from the DataFrame
y = data['sales']

# equivalent command that works if there are no spaces in the column name
y = data.sales

# print the first 5 values
y.head()

In [ ]:
# check the type and shape of y
print(type(y))
print(y.shape)

## 将X和y划分成训练集和测试集

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

## 做线性回归

In [ ]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

## 解释模型的系数

In [ ]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

In [ ]:
# pair the feature names with the coefficients
list(zip(feature_cols, linreg.coef_))

$y = 2.89 + 0.044 \times TV + 0.199 \times Radio + 0.001 \times Newspaper$

如何解释TV的系数，即0.044呢？

给定广播广告费用和报纸广告费用，每增加一个单位的电视广告费，会增加0.044个单位的销售量。比如，每增加1000美金的电视广告费用，会多销售44件商品。

如果系数是负数，则表示销售量会随着广告费用的增加而降低。

## 预测并评价

In [ ]:
# make predictions on the testing set
y_pred = linreg.predict(X_test)

考察回归模型常用的三种评价指标的计算方法：MAE, MSE, RMSE：

In [ ]:
# define true and predicted response values
true = [100, 50, 30, 20]
pred = [90, 50, 50, 30]

In [ ]:
# calculate MAE by hand
print((10 + 0 + 20 + 10)/4.)

# calculate MAE using scikit-learn
from sklearn import metrics
print(metrics.mean_absolute_error(true, pred))

In [ ]:
# calculate MSE by hand
print((10**2 + 0**2 + 20**2 + 10**2)/4.)

# calculate MSE using scikit-learn
print(metrics.mean_squared_error(true, pred))

In [ ]:
# calculate RMSE by hand
import numpy as np
print(np.sqrt((10**2 + 0**2 + 20**2 + 10**2)/4.))

# calculate RMSE using scikit-learn
print(np.sqrt(metrics.mean_squared_error(true, pred)))

实际计算各评价指标在测试集上预测的结果：

In [ ]:
print("Mean absolute error (MAE):", metrics.mean_absolute_error(y_test,y_pred))
print("Mean square error (MSE):", metrics.mean_squared_error(y_test,y_pred))
print("Root mean square error (RMSE):", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print("R-squared value of predictions:",round(metrics.r2_score(y_test,y_pred),3))

## 交叉验证

In [ ]:
from sklearn.model_selection import cross_val_score
r2_scores = cross_val_score(linreg, X, y, cv=10)
mse_scores = cross_val_score(linreg, X, y, cv=10, scoring='neg_mean_squared_error')

In [ ]:
print(r2_scores)
print(mse_scores)

In [ ]:
print(r2_scores.mean())
print(mse_scores.mean())

## 特征选择
“newspaper”，报纸广告费用，是否应该加入模型，即是否对预测销售量有帮助？

In [ ]:
# create a Python list of feature names
feature_cols = ['TV', 'radio']

# use the list to select a subset of the original DataFrame
X = data[feature_cols]

# select a Series from the DataFrame
y = data.sales

# split into training and testing sets
r2_scores = cross_val_score(linreg, X, y, cv=10)
mse_scores = cross_val_score(linreg, X, y, cv=10,scoring='neg_mean_squared_error')
print(r2_scores.mean())
print(mse_scores.mean())

去掉“newspaper”特征，RMSE没发生任何变化，因而可以从模型中去掉该特征

## 增加高次特征，分别做线性回归、岭回归和套索回归

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=2)
X_train_poly = pf.fit_transform(X_train)
X_test_poly = pf.fit_transform(X_test)

In [ ]:
X_train_poly

In [ ]:
lr = LinearRegression()
lr.fit(X_train_poly, y_train)
y_pred = lr.predict(X_test_poly)
print("Root mean square error (RMSE):", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

In [ ]:
from sklearn.linear_model import Ridge, Lasso

# The ridge regression model
rr = Ridge(alpha=0.001)
rr.fit(X_train_poly, y_train)
y_pred_rr = rr.predict(X_test_poly)
print("Root mean square error (RMSE):", np.sqrt(metrics.mean_squared_error(y_test,y_pred_rr)))

# The lasso regression model
lassor = Lasso(alpha=0.0001)
lassor.fit(X_train_poly, y_train)
y_pred_lr = lassor.predict(X_test_poly)
print("Root mean square error (RMSE):", np.sqrt(metrics.mean_squared_error(y_test,y_pred_lr)))

## 分别使用RidgeCV, LassoCV和ElasticNetCV在某个范围内自动寻找最佳正则化参数

In [ ]:
from sklearn.linear_model import RidgeCV

alphas = [0.005, 0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 80]

ridgeCV = RidgeCV(alphas=alphas, cv=10).fit(X_train, y_train)

y_pred = ridgeCV.predict(X_test)

print("Root mean square error (RMSE):", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print(ridgeCV.alpha_)

In [ ]:
from sklearn.linear_model import LassoCV

alphas2 = np.array([1e-5, 5e-5, 0.0001, 0.0005])

lassoCV = LassoCV(alphas=alphas2, max_iter=5e4, cv=10).fit(X_train, y_train)

y_pred = lassoCV.predict(X_test)

print("Root mean square error (RMSE):", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print(lassoCV.alpha_)

In [ ]:
from sklearn.linear_model import ElasticNetCV

l1_ratios = np.linspace(0.1, 0.9, 9)

elasticNetCV = ElasticNetCV(alphas=alphas2, l1_ratio=l1_ratios, max_iter=1e4, cv=10).fit(X_train, y_train)

y_pred = elasticNetCV.predict(X_test)

print("Root mean square error (RMSE):", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print(elasticNetCV.alpha_, elasticNetCV.l1_ratio_)